# Data Processing and Ingestion

## Introduction

This notebook handles various input data types, transforming them into a document format, and then incorporates them into a vector database for subsequent processing. The indexed documents are further used in the next notebook to create an AI service function and deploy it on [watsonx.ai](https://www.ibm.com/products/watsonx-ai) . 

The initial section of the notebook focuses onfocuses on handling any type of data contained within a file, directory or a zip archive. The process involves extracting content from different file types, segmenting the data, converting it into a document format, and finally indexing the content into a vector database. 

The accelerator currently supports **Elasticsearch, Milvus and Datastax** vector databases.

The ingestion process uses vector embeddings to enhance data storage and retrieval within either Elasticsearch or Milvus or Datastax vector databases, ensuring both efficiency and effectiveness. 

- Establishing a connection to the chosen vector database (Elasticsearch or Milvus or Datastax) and loading input data from processed documents.
- Generating unique IDs for documents and splitting them into manageable chunks for indexing in the vector database.
- Inserting the documents with embeddings in batches into Elasticsearch or Milvus or Datastax using these generated IDs, with progress monitoring provided by a progress bar.
- When using a Milvus connection, the code sets up a Milvus vector store with dense or hybrid (dense + sparse) embeddings based on the search type. 
- When using a Datastax connection, the code sets up a Datastax vector store with dense embeddings. 

**Note**: 
- It is recommended to run this notebook in a Python environment on watsonx.ai software with a GPU-enabled or high vCPU and RAM hardware configuration, as generating embeddings may require significant memory.
- Datastax is not supported in this cloud version.

## Contents

This notebook contains the following parts:
- [Setup](#setup)
- [Import Dependencies](#import)
- [Extract Data from Input files](#input)
- [Process and Split Extracted Data](#split)
- [Connect to Vector Database](#connect)
- [Index Documents using langchains vectorstore](#insert_documents)


<a id="setup"></a>
### Pre-Requisite Libraries and Dependencies
Below cell downloads and installs specific mandatory libraries and dependencies required to run this notebook.

**Note** : Some of the versions of the libraries may throw warnings after installation. These library versions are crucial for execution of the accelerator. Please ignore the warning/error and proceed with your execution. 

In [ ]:
!pip install elasticsearch==8.18.1 | tail -n 1
!pip install langchain_community | tail -n 1
!pip install unstructured==0.17.2 | tail -n 1
!pip install langchain | tail -n 1
!pip install ibm_watsonx_ai==1.3.26| tail -n 1
!pip install langchain_elasticsearch==0.3.2 | tail -n 1
!pip install langchain_milvus==0.2.0 | tail -n 1
!pip install pymilvus==2.5.11 | tail -n 1
!pip install tiktoken | tail -n 1
!pip install pypdf | tail -n 1
!pip install cassio==0.1.10 | tail -n 1

Restart the kernel after performing the pip install if the below cell fails to import all the libraries.

In [ ]:
from elasticsearch import Elasticsearch, helpers
from ibm_watsonx_ai import Credentials
from ibm_watsonx_ai.foundation_models import Embeddings
from ibm_watsonx_ai.metanames import EmbedTextParamsMetaNames as EmbedParams
from ibm_watsonx_ai.foundation_models.utils.enums import EmbeddingTypes
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams
from ibm_watsonx_ai import APIClient

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import DirectoryLoader,PyPDFLoader,UnstructuredHTMLLoader

import hashlib
from bs4 import BeautifulSoup
import multiprocessing
import json
import os
import asyncio
import re
import zipfile
from threading import Thread
import shutil 
import warnings
from tqdm import tqdm
import time

from pymilvus import(IndexType,Status,connections,FieldSchema,DataType,Collection,CollectionSchema,utility)
from langchain_milvus import Milvus, BM25BuiltInFunction
warnings.filterwarnings("ignore")

In [ ]:
project_id=os.environ['PROJECT_ID']
# Environment and host url
hostname = os.environ['RUNTIME_ENV_APSX_URL']

if hostname.endswith("cloud.ibm.com") == True:
    environment = "cloud"
    project_id = os.environ['PROJECT_ID']
    runtime_region = os.environ["RUNTIME_ENV_REGION"] 
else:
    environment = "on-prem"
    from ibm_watson_studio_lib import access_project_or_space
    wslib = access_project_or_space()   

<a id="import"></a>
### Import Parameter Sets, Credentials and Helper functions script.

Below cells imports parameter sets values, sets the watsonx.ai credentials and imports the helper functions script. 

In [ ]:
try:
    filename = 'rag_helper_functions.py'
    wslib.download_file(filename)
    import rag_helper_functions
    print("rag_helper_functions imported from the project assets")
except NameError as e:
    print(str(e))
    print("If running watsonx.ai aaS on IBM Cloud, check that the first cell in the notebook contains a project token. If not, select the vertical ellipsis button from the notebook toolbar and `insert project token`. Also check that you have specified your ibm_api_key in the second code cell of the notebook")


In [ ]:
parameter_sets = ["RAG_parameter_set","RAG_advanced_parameter_set"]

parameters=rag_helper_functions.get_parameter_sets(wslib, parameter_sets)

In [ ]:
ibm_api_key=parameters['watsonx_ai_api_key']
if environment == "cloud":
    WML_SERVICE_URL = f"https://{runtime_region}.ml.cloud.ibm.com"
    wml_credentials = Credentials(api_key=parameters['watsonx_ai_api_key'], url=WML_SERVICE_URL)
else:
    token = os.environ['USER_ACCESS_TOKEN']
    wml_credentials=Credentials(token=os.environ['USER_ACCESS_TOKEN'],url=hostname,instance_id='openshift')

### Set Watsonx.ai client
Below cell uses the watson machine learning credentials to create an API client to interact with the project and deployment space. 

In [ ]:
client = APIClient(wml_credentials)
client.set.default_project(project_id=project_id)

<a id="input"></a>

### Input File for extracting, loading, processing and indexing into a vector database. 

This can be updated in the **RAG parameter set** as required. Here are some of the options below on the formats that can be used:
- The name of a zip file containing **.pdf / .docx / .pptx / .html/ .md/ .txt** files that exists in the project as a data asset. e.g. `ibm-docs-SSYOK8.zip`
- The name of a **.pdf / .docx / .pptx / .html / .md/ .txt** file that exists in the project as a data asset. 


In [ ]:
input_filename = parameters['ingestion_data_file']   
    
if any(input_filename.lower().endswith(ext) for ext in ['.zip', '.pdf', '.docx', '.pptx', '.html',".md", ".txt"]):
    wslib.download_file(input_filename)
else:
    raise ValueError("Input document data asset should have an extension (.zip/.pdf/.docx/.pptx/.html/.md/.txt) only!")
    

if the format of the file is `.zip` then below cell extracts the files in the zip into a directory.

In [ ]:
directory ="watson-docs"

if os.path.exists(directory):
    shutil.rmtree(directory)
os.makedirs(directory)
if ".zip" in input_filename:
    with zipfile.ZipFile(input_filename, 'r') as zip_ref:
        zip_ref.extractall(path=directory)

    print("Extraction completed!")
else:
    shutil.move(input_filename, directory)
    print("File copied")


<a id="split"></a>
### Load all the Documents 

The following cell generates a `DirectoryLoader` instance from the langchain library, configured to read files with extensions such as `.html, .pdf, .pptx, .md, .txt and .docx.` The loader traverses the specified folder, gathers all the documents, and appends them to a list. 

In the subsequent cell, these documents are processed to be inserted into a vector database. This involves splitting the documents using langchain's `RecursiveCharacterTextSplitter` and incorporating both content and metadata into the documents. The term "recursive" suggests that this division process happens in multiple stages or levels, breaking down the text into increasingly smaller segments.

In [ ]:
import nltk
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger_eng')
loaders = [
    DirectoryLoader(directory, glob="**/*.html",show_progress=True),
    DirectoryLoader(directory, glob="**/*.pdf",show_progress=True, loader_cls=PyPDFLoader),
    DirectoryLoader(directory, glob="**/*.pptx",show_progress=True),
    DirectoryLoader(directory, glob="**/*.docx",show_progress=True),
    DirectoryLoader(directory, glob="**/*.md",show_progress=True),
    DirectoryLoader(directory, glob="**/*.txt",show_progress=True)    
]


documents=[]
for loader in loaders:
    data =loader.load()
    documents.extend(data)

The cell below prepares documents for insertion into a vector database. It handles HTML files where document URLs are stored within the `canonical` tag. The cell reads the `canonical` tag and incorporates it into the HTML files metadata.

In [ ]:
def get_split_documents(documents):
    content=[]
    metadata = []
    for doc in documents:
        
        document_url = ""
        document_title = doc.metadata["title"] if "title" in doc.metadata else doc.metadata['source'].split("/")[-1].split(".")[0]
    
        if ".html" in doc.metadata['source']:
            with open(doc.metadata['source'], 'r', encoding='utf-8') as html_file:
                html_content = html_file.read()
            soup = BeautifulSoup(html_content, 'html.parser')
            canonical_tag = soup.find('link', {'rel': 'canonical'})
            title_tag = soup.find('title')
    
            if canonical_tag:
                document_url = canonical_tag.get('href')
            
            if title_tag:
                document_title=title_tag.get_text()
        
        metadata.append({
                "title":document_title ,
                "source": doc.metadata['source'],
                "document_url":document_url,
                "page_number":str(doc.metadata['page']) if 'page' in doc.metadata else ''
                
            })
        
        content.append(doc.page_content)
    
    text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
        chunk_size=parameters['ingestion_chunk_size'],
        chunk_overlap=parameters['ingestion_chunk_overlap'],
        disallowed_special=()
    )
    
    split_documents = text_splitter.create_documents(content, metadatas=metadata)
    print(f"{len(documents)} Documents are split into {len(split_documents)} documents with a chunk size",parameters["ingestion_chunk_size"])
    
    
    
    for chunk in split_documents:
        chunk.metadata["title"] = chunk.metadata.get("title", "Unknown Title")
        chunk.page_content = f"Document Title: {chunk.metadata['title']}\n Document Content: {chunk.page_content}"
    
    split_docs = rag_helper_functions.remove_duplicate_records(split_documents)
    print(f'After de-duplication, there are {len(split_docs)} documents present')

    return split_docs

In [ ]:
split_docs = get_split_documents(documents)

<a id="connect"></a>
### Connecting to a vector database

#### Connecting using Project Connection Asset (default)

The notebook, by default, will look for a connection asset in the project named `milvus_connect` or `elasticsearch_connect` or `datastax_connect`.  You can set this up by following the instructions in the project readme. 
This code checks if a specified connection exists in the project. If found, it retrieves the connection details and identifies the connection type. Depending on the connection type, it establishes a connection to the appropriate database. If the connection is not found, it raises an error indicating the absence of the specified connection in the project.

Additionally, 
If the connection type is **Elastic Search**, the below cell gets the status of the current Elasticsearch model (e.g. ELSER 2). For this ensure that you have the model downloaded and deployed on Elasticsearch. This can be done under **Machine Learning >> Trained Models** section on Elasticsearch.

If the connection type is **DataStax**, the below cell gets the keyspace if not exists, it will create the keyspace with SimpleStrategy with single datacenter. For `Production` usecases, it is **recommended** to create your own `keyspace strategy` based on your network configuration which may need to apply custom changes in `datastax_ks_replication`. Also update cluster connection pooling as per official [doc](https://docs.datastax.com/en/datastax-drivers/connecting/connection-pool.html) for further customisations on its usage.


In [ ]:
connection_name=parameters["connection_asset"]
if(next((conn for conn in wslib.list_connections() if conn['name'] == connection_name), None)):
    print(connection_name, "Connection found in the project")
    db_connection = wslib.get_connection(connection_name)
    
    connection_datatypesource_id=client.connections.get_details(db_connection['.']['asset_id'])['entity']['datasource_type']
    connection_type = client.connections.get_datasource_type_details_by_id(connection_datatypesource_id)['entity']['name']
    
    print("Successfully retrieved the connection details")
    print("Connection type is identified as:",connection_type)

    if connection_type=="elasticsearch":
        es_client=rag_helper_functions.create_and_check_elastic_client(db_connection, parameters['elastic_search_model_id'])
    elif connection_type=="milvus" or connection_type=="milvuswxd":
        milvus_credentials = rag_helper_functions.connect_to_milvus_database(db_connection, parameters)
    elif connection_type=="datastax":
        if environment == "cloud":
            raise ValueError(f"ERROR! we don't support officially datastax for Cloud as of now")
        datastax_session,datastax_cluster = rag_helper_functions.connect_to_datastax(db_connection, parameters)
        try:
            if datastax_session==None:
                raise ValueError(f"Failure in connecting named {connection_name} found in the project. Please check above")
            else:
                if 'keyspace' not in db_connection:
                    raise ValueError(f"Failure in connecting named {connection_name} found in the project. Please add keyspace which is missing!")
                import cassio
                check_datastax_ks_exists = rag_helper_functions.check_datastax_ks_exists(db_connection, datastax_session, client, parameters)
                # Initialize cassio with the session and keyspace
                cassio.init(session=datastax_session, keyspace=db_connection.get('keyspace'))
        
                if not check_datastax_ks_exists:
                    # It is recommended to create your Keyspace by checking with Datastax Cluster administrator if you don't have already keyspace & use.
                    raise ValueError(f"ERROR: keyspace not found, Please create keyspace by checking with your administrator")
        except Exception as e:
            print(f"Failed in Datastax Connection {e}")
else:
    db_connection=""
    raise ValueError(f"No connection named {connection_name} found in the project.")

<a id="insert_documents"></a>
### Inserting Documents using Langchains Vector Store 

Below code utilizes Langchains vector store extension to store documents. <br>
The code sets up a vector store based on the specified connection type, either **"elasticsearch" or "milvus"**. <br>

- If `connection_type` is `"elasticsearch"`, it imports `ElasticsearchStore` from the `langchain_elasticsearch` library and initializes it with an Elasticsearch client and specified parameters with the given model ID. The `Elastic Search`vector store is then created using the `model_id`, connection parameters and index settings <br>

- If `connection_type` is `"milvus"`, the code imports `langchain_milvus` and configures credentials using an API key and service URL. It initializes the embedding model specified in the parameters to generate embeddings, and sets up index parameters with specific metrics and configurations. The `Milvus` vector store is then created using the embedding function, connection details, and index settings. For both cloud & SW environments, it uses the `ibm_watsonx_ai` library to initialize the embedding function with the required API key, URL, and project ID. On Cloud Pak for Data software, required encoded models needs to deployed which is mentioned in parameters["embedding_model_id"] if they are not present on the cluster. Once loaded, those models are used to generate embeddings for the chunked documents. It initializes a **Milvus** vector store with either **dense embeddings** or **hybrid search** (dense + BM25 sparse embeddings) based on the `milvus_hybrid_search` parameter. If hybrid search is enabled, it creates a new collection with **IVF_FLAT for dense** and **BM25 for sparse** indexing. Otherwise, it only adds **dense embeddings**. 

* If `connection_type` is `"datastax`", it uses `langchain_community.vectorstores.Cassandra` to initialize a **Cassandra-based vector store**. The embedding model is initialized similarly using `get_embedding()`, and the store is configured using a specified keyspace and table name. This setup supports vector search on a DataStax Astra DB or Cassandra cluster.

Firstly, it creates a connection to the vector database and defines how documents will be retrieved later. <br>
Then, it defines a function to add these documents to the vector store. This function takes the documents and additional parameters for efficient processing, such as splitting the documents into smaller chunks and setting a timeout for requests. <br>

Overall, this code efficiently adds a list of documents to a vector store, thereby making them searchable. <br>

Below code creates the `Elasticsearch` dense vector index for indexing of documents if dense embedding model like `E5 multilingual` is used and also creates elasticsearch pipeline for indexing.

In [ ]:
def create_es_dense_index(index_name):
    try:
        es_client.options(ignore_status=400).indices.create(
                    index=index_name,
                    mappings={
                        'properties': {
                            'vector.tokens': {
                                'type': 'dense_vector',
                            },
                        }
                    },
                    settings={
                        'index': {
                            'default_pipeline': 'dense-ingest-pipeline',
                        },
                        "number_of_shards": parameters["es_number_of_shards"],
                    }
        )
        es_client.ingest.put_pipeline(
                    id='dense-ingest-pipeline',
                    processors=[
                        {
                            'inference': {
                                'model_id': parameters['elastic_search_model_id'],
                                'input_output': [
                                    {
                                        'input_field': 'text',
                                        'output_field': 'vector.tokens',
                                    }
                                ]
                            }
                        }
                    ]
                )
        print(f'Elastic search index created with {parameters["elastic_search_model_id"]}!')
    except Exception as e:
        print('Error creating elastic index', e)

In [ ]:

def get_embedding(environment, parameters, project_id, wml_credentials, WML_SERVICE_URL):
    if environment == "cloud":
        credentials = Credentials(
            api_key=parameters['watsonx_ai_api_key'],
            url=WML_SERVICE_URL
        )
        embedding = Embeddings(
            model_id=parameters['embedding_model_id'],
            credentials=credentials,
            project_id=project_id,
            verify=True
        )
    elif environment == "on-prem":
        try:
            if client.foundation_models.EmbeddingModels.__members__:
                if client.foundation_models.EmbeddingModels(parameters["embedding_model_id"]).name:
                    embedding = Embeddings(
                        model_id=parameters['embedding_model_id'],
                        credentials=wml_credentials,
                        project_id=project_id,
                        verify=True
                    )
                else:
                    print(f"Encoder model {parameters['embedding_model_id']} not found on the cluster. Please update embedding model_id param to a model which exists or deploy missing model on this cluster")
            else:
                print("Local on-prem embedding models not found, using models from IBM Cloud API if required parameters are provided")
                credentials = Credentials(
                    api_key=parameters['watsonx_ai_api_key'],
                    url=parameters['watsonx_ai_url']
                )
                embedding = Embeddings(
                    model_id=parameters['embedding_model_id'],
                    credentials=credentials,
                    space_id=parameters["wx_ai_inference_space_id"],
                    verify=True
                )
        except Exception as e:
            print(f"Exception in loading Embedding Models: {str(e)}")
            raise
    else:
        raise ValueError(f"Invalid environment: {environment}. Must be 'cloud' or 'on-prem'.")
    
    return embedding

In [ ]:
def create_vector_store(connection_type,index_name,parameters):
    if connection_type=="elasticsearch":
        from langchain_elasticsearch import ElasticsearchStore
        if 'dense' in parameters['elastic_search_vector_type']:
            create_es_dense_index(index_name)
            vector_store=ElasticsearchStore(
                            es_connection=es_client,
                            index_name=index_name,
                            strategy=ElasticsearchStore.ApproxRetrievalStrategy(query_model_id=parameters['elastic_search_model_id']),
                            )
        else:
            vector_store=ElasticsearchStore(
                            es_connection=es_client,
                            index_name=index_name,
                            strategy=ElasticsearchStore.SparseVectorRetrievalStrategy(model_id=parameters['elastic_search_model_id']),
                            custom_index_settings={"number_of_shards": parameters["es_number_of_shards"]}
                            )
        print("Elastic Search Vector Store Created with",parameters['elastic_search_model_id'])
    elif connection_type=="milvus" or connection_type=="milvuswxd":
        from langchain_milvus import Milvus
        print("using the model",parameters['embedding_model_id'], "to create embeddings")
        embedding = get_embedding(environment, parameters, project_id, wml_credentials, WML_SERVICE_URL) if environment == "cloud" else get_embedding(environment, parameters, project_id, wml_credentials, None)  
        #milvus_index_params = {"index_type": "HNSW","metric_type": "L2",  "params": { "M": 16,"efConstruction": 200,"efSearch": 16 }}
        dense_index_param = {"metric_type": "L2", "index_type": "IVF_FLAT","params": {"nlist": 1024},}
        sparse_index_param = {"metric_type": "BM25","index_type": "SPARSE_INVERTED_INDEX", "params": {"drop_ratio_build": 0.2}}

        hybrid_search = True if parameters['milvus_hybrid_search'].lower()=="true" else False
        if hybrid_search:
            print("Adding sparse and Dense embeddings")
            vector_store = Milvus(
            embedding_function=embedding,
            builtin_function=BM25BuiltInFunction(output_field_names="sparse"), 
            index_params=[dense_index_param, sparse_index_param],
            vector_field=["dense", "sparse"],
            connection_args=milvus_credentials,
            primary_field='id',
            consistency_level="Strong",
            collection_name=index_name
            )
        else:
            print("Adding Dense embeddings")
            vector_store = Milvus(
                embedding_function=embedding,
                index_params=dense_index_param,
                connection_args=milvus_credentials,
                primary_field='id',
                consistency_level="Strong",
                collection_name=index_name
            )
        print("Milvus Vector Store Created")

    elif connection_type == "datastax":
        if environment == "cloud":
            raise ValueError(f"ERROR! we don't support datastax connection for Cloud as of now")
        print("using the model",parameters['embedding_model_id'], "to create embeddings")
        embedding = get_embedding(environment, parameters, project_id, wml_credentials, WML_SERVICE_URL) if environment == "cloud" else get_embedding(environment, parameters, project_id, wml_credentials, None)  
        from langchain_community.vectorstores import Cassandra
        vector_store = Cassandra(
            embedding=embedding,
            table_name=index_name
        )
        print("Datastax Vector Store Created")

    return vector_store

In [ ]:
def generate_hash(content):
    return hashlib.sha256(content.encode()).hexdigest()


def insert_docs_to_vector_store(vector_store,split_docs,insert_type="docs" ):
    with tqdm(total=len(split_docs), desc="Inserting Documents", unit="docs") as pbar:
        try:
            for i in range(0, len(split_docs), parameters['index_chunk_size']):
                chunk = split_docs[i:i + parameters['index_chunk_size']]
                if insert_type=="docs":
                    id_chunk = [generate_hash(doc.page_content+'\nTitle: '+doc.metadata['title']+'\nUrl: '+doc.metadata['document_url']+'\nPage: '+doc.metadata['page_number']) for doc in chunk]
                elif insert_type=="profiles":
                    id_chunk = [generate_hash(doc.page_content) for doc in chunk]
                vector_store.add_documents(chunk, ids=id_chunk)
                pbar.update(len(chunk))
            print("Documents are inserted into vector database")
        except Exception as e:
            print(f"An error occurred: {e}")

Below code defines a function to generates a list of unique IDs for each document by hashing their `page_content`. The code sets a chunk size for batch processing. It iterates over the documents in chunks, inserting each chunk into the vector store with corresponding IDs. The progress bar is updated to reflect the number of documents processed.

In [ ]:
vector_store=create_vector_store(connection_type,parameters['vector_store_index_name'], parameters)
print("Inserting Documents") 
insert_docs_to_vector_store(vector_store,split_docs,"docs")

Above cell may take significant amount of time to complete based on the size of the documents. 

Optionally you can also proceed to : 
* **Test Queries for Vector Database** notebook to perform sample searches using various techniques and Analyze the search results to understand the effectiveness of different methods. 
* **Ingest Expert Profile data to vector DB** notebook to ingest expert profile data into the vector database. 

If you do not wish to **test sample search queries against the vector database** or **ingest expert profile data to the vector database**, you can proceed to **`Create and Deploy QnA AI Service`** notebook to create and deploy the RAG AI service without waiting for the next cell to complete.<br>

Below cell can be uncommented to make the above insertion asynchronous.

In [ ]:
#vector_store=create_vector_store(connection_type,parameters['vector_store_index_name'], parameters)
#documents=asyncio.create_task(
#    vector_store.add_documents(
#    split_docs, 
#    bulk_kwargs={
#                "chunk_size": parameters['index_chunk_size'],
#                "request_timeout": 600
#            },ids=[generate_hash(doc.page_content+'\nTitle: '+doc.metadata['title']+'\nUrl: '+doc.metadata['document_url']+'\nPage: '+doc.metadata['page_number']) for doc in split_docs])
#)

**Note** It's recommended to close the datastax session once you are done with ingestion in this notebook for optimal performance. once you execute this cell existing datastax connections are closed. if have to re run above code cells you have to create new connection for datastax by re running cells from `Connect to Vector Database`

In [ ]:
if connection_type=="datastax" and environment != "cloud":
    if not datastax_session.is_shutdown:
        datastax_session.shutdown()
        print(f"datastax_session got shutdown : {datastax_session.is_shutdown}")
    if not datastax_cluster.is_shutdown:
        datastax_cluster.shutdown()
        print(f"datastax_cluster got shutdown : {datastax_cluster.is_shutdown}")



**Sample Materials, provided under license.</a> <br>
Licensed Materials - Property of IBM. <br>
© Copyright IBM Corp. 2024, 2025. All Rights Reserved. <br>
US Government Users Restricted Rights - Use, duplication or disclosure restricted by GSA ADP Schedule Contract with IBM Corp. <br>**
